# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 1. Create Label Map

In [2]:
labels = [
    {'name':'Hello', 'id':1}, 
    {'name':'yes', 'id':2},
    {'name':'No', 'id':3},
    {'name':'thank you', 'id':4},
    {'name':'I love you', 'id':5},
         
]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [3]:
import sys
import subprocess
from pathlib import Path

def run(cmd):
    print("\n$", " ".join(map(str, cmd)))
    p = subprocess.run(cmd, text=True, capture_output=True)
    if p.stdout:
        print(p.stdout)
    if p.stderr:
        print(p.stderr, file=sys.stderr)
    p.check_returncode()

script_path = str((Path(SCRIPTS_PATH) / "generate_tfrecord.py").resolve())
label_map_path = str((Path(ANNOTATION_PATH) / "label_map.pbtxt").resolve())

train_dir = str((Path(IMAGE_PATH) / "train").resolve())
test_dir = str((Path(IMAGE_PATH) / "test").resolve())
train_record = str((Path(ANNOTATION_PATH) / "train.record").resolve())
test_record = str((Path(ANNOTATION_PATH) / "test.record").resolve())

print("Using Python:", sys.executable)
print("Label map:", label_map_path)

run([sys.executable, script_path, "-x", train_dir, "-l", label_map_path, "-o", train_record])
run([sys.executable, script_path, "-x", test_dir, "-l", label_map_path, "-o", test_record])

Using Python: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\.venv\Scripts\python.exe
Label map: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\label_map.pbtxt

$ C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\.venv\Scripts\python.exe C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\scripts\generate_tfrecord.py -x C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\images\train -l C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\label_map.pbtxt -o C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\train.record
Successfully created the TFRecord file: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\train.record


$ C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\.venv\Scripts\python


[generate_tfrecord] Added missing label 'ww' to C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\label_map.pbtxt with id 6.



Successfully created the TFRecord file: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\annotations\test.record
Skipped 2 examples due to missing image files.



# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [4]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [5]:
#wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
#!mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
#!cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [6]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [7]:
from pathlib import Path
import shutil

# Windows-safe replacement for creating the model folder + copying pipeline.config
dst_dir = (Path(MODEL_PATH) / CUSTOM_MODEL_NAME).resolve()
dst_dir.mkdir(parents=True, exist_ok=True)

src_cfg = (Path(PRETRAINED_MODEL_PATH) / 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8' / 'pipeline.config').resolve()
dst_cfg = dst_dir / 'pipeline.config'

if not src_cfg.exists():
    raise FileNotFoundError(f'pipeline.config not found: {src_cfg}')

shutil.copyfile(src_cfg, dst_cfg)
print(f'Model dir: {dst_dir}')
print(f'Copied: {src_cfg} -> {dst_cfg}')


Model dir: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\models\my_ssd_mobnet
Copied: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\pre-trained-models\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\pipeline.config -> C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config


# 5. Update Config For Transfer Learning

In [8]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [9]:
from pathlib import Path
import sys

# Ensure we import the Object Detection API from THIS repo checkout (Windows-safe).
cwd = Path.cwd().resolve()
research_path = None
for parent in [cwd, *cwd.parents]:
    candidate = parent / "Tensorflow" / "models" / "research"
    if candidate.exists():
        research_path = candidate
        break

if research_path is None:
    raise FileNotFoundError(
        "Could not find Tensorflow/models/research under the current working directory. "
        "Open the notebook from the repo root, or adjust the path logic here."
    )

slim_path = research_path / "slim"
sys.path.insert(0, str(research_path))
sys.path.insert(0, str(slim_path))

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [10]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [11]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [12]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-005
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     override_base_feature_extractor_hyperparams: true
     use_depthwise: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10
       x_scale: 10
       height_scale: 5
       width_scale: 5
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatch

In [13]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [14]:
pipeline_config.model.ssd.num_classes = 5
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [15]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [16]:
import os
import subprocess
import sys
from pathlib import Path

def _find_repo_root(start: Path) -> Path:
    for parent in [start, *start.parents]:
        if (parent / "Tensorflow" / "workspace").exists():
            return parent
    return start

repo_root = _find_repo_root(Path.cwd().resolve())

# Fall back to defaults if earlier setup cells weren't run yet.
custom_model_name = globals().get("CUSTOM_MODEL_NAME", "my_ssd_mobnet")
workspace_path = Path(globals().get("WORKSPACE_PATH", repo_root / "Tensorflow" / "workspace")).resolve()
model_root = Path(globals().get("MODEL_PATH", workspace_path / "models")).resolve()

# Use the Object Detection API checkout in this repo.
research_path = Path(globals().get("research_path", repo_root / "Tensorflow" / "models" / "research")).resolve()
slim_path = Path(globals().get("slim_path", research_path / "slim")).resolve()

model_main = research_path / "object_detection" / "model_main_tf2.py"
model_dir = (model_root / custom_model_name).resolve()
pipeline_cfg = model_dir / "pipeline.config"

env = os.environ.copy()
env["PYTHONPATH"] = os.pathsep.join([str(research_path), str(slim_path)])

cmd = [
    sys.executable,
    str(model_main),
    f"--model_dir={model_dir}",
    f"--pipeline_config_path={pipeline_cfg}",
    "--num_train_steps=10000",
    # "--num_workers=0",  # can help on Windows if multiprocessing causes issues
    # "--alsologtostderr",
 ]

print("Repo root:", repo_root)
print("Using Python:", sys.executable)
print("Training script:", model_main)
print("Model dir:", model_dir)
print("Pipeline config:", pipeline_cfg)
print("PYTHONPATH:", env["PYTHONPATH"])
if not model_main.exists():
    print("WARNING: model_main_tf2.py not found at:", model_main)
if not pipeline_cfg.exists():
    print("WARNING: pipeline.config not found at:", pipeline_cfg)
print()
print("$ " + " ".join(cmd))

# Uncomment to actually start training from the notebook (it can run a long time).
# subprocess.run(cmd, env=env, check=True)

Repo root: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter
Using Python: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\.venv\Scripts\python.exe
Training script: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\models\research\object_detection\model_main_tf2.py
Model dir: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\models\my_ssd_mobnet
Pipeline config: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\workspace\models\my_ssd_mobnet\pipeline.config
PYTHONPATH: C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\models\research;C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\models\research\slim

$ C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\.venv\Scripts\python.exe C:\Users\nus\Documents\GitHub\American-sign-language-interpreter\Tensorflow\models\research\object_detection\model_main_tf2.

# 7. Load Train Model From Checkpoint

In [17]:
import os
import sys
from pathlib import Path
import numpy as np

# Ensure Object Detection API imports come from this repo checkout, even after a kernel restart.
if "research_path" not in globals() or "slim_path" not in globals():
    cwd = Path.cwd().resolve()
    research_path = None
    for parent in [cwd, *cwd.parents]:
        candidate = parent / "Tensorflow" / "models" / "research"
        if candidate.exists():
            research_path = candidate
            break
    if research_path is None:
        raise FileNotFoundError("Could not find Tensorflow/models/research under the current working directory")
    slim_path = research_path / "slim"
    sys.path.insert(0, str(research_path))
    sys.path.insert(0, str(slim_path))

import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [18]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [19]:
import cv2
cap = cv2.VideoCapture(0)
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
print("Camera Reset Successful")

Camera Reset Successful


In [20]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [21]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    ret, frame = cap.read()
    
    # SAFETY CHECK: If the camera doesn't return a frame, skip this loop
    if not ret or frame is None:
        continue
        
    image_np = np.array(frame)
    
    # Convert image to a tensor for the model
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    # Process detections
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Detection classes must be integers
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    # Draw boxes and labels on the frame
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    # Show the window
    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    # PRESS 'q' TO CLOSE THE WINDOW
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

In [ ]:
detections = detect_fn(input_tensor)

In [ ]:
from matplotlib import pyplot as plt